In [8]:
import numpy as np
import torch
import torch.nn as nn

input_size = 7
batch_size = 5
input_tensor = torch.randn(batch_size, input_size, dtype=torch.float)

eps = 1e-3
batch_norm = nn.BatchNorm1d(input_size, eps=eps)
batch_norm.bias.data = torch.randn(input_size, dtype=torch.float)
batch_norm.weight.data = torch.randn(input_size, dtype=torch.float)

# Функция для нормализации батча
def custom_batch_norm1d(input_tensor, weight, bias, eps):
    # среднее по стобцам (по одним эл-там всех векторов)
    mean = torch.mean(input_tensor, 0, keepdim=True)
    # отклонение(дисперсия) по стобцам (по одним эл-там всех векторов)
    std = torch.std(input_tensor, 0, keepdim=True, unbiased=False)
    # нормированные тензор
    normed_tensor = ((input_tensor - mean)/torch.sqrt(std**2+eps)) * weight + bias
    return normed_tensor

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
batch_norm_out = batch_norm(input_tensor)
custom_batch_norm_out = custom_batch_norm1d(input_tensor, batch_norm.weight.data, batch_norm.bias.data, eps)

print(torch.allclose(batch_norm_out, custom_batch_norm_out) \
      and batch_norm_out.shape == custom_batch_norm_out.shape)

True


In [13]:
# Класс для батч нормализации
import torch
import torch.nn as nn

input_size = 3
batch_size = 5
eps = 1e-1

class CustomBatchNorm1d:
    # Реализуйте в этом месте конструктор.
    def __init__(self, weight, bias, eps, momentum):
        self.weight = weight
        self.bias = bias
        self.eps = eps
        self.momentum = momentum
        self.sigma = torch.ones(input_size)
        self.mean = torch.zeros(input_size)
        self.evaluation = False

    # Напишите в этом месте нормирование входного тензора.
    def __call__(self, input_tensor):
        batch_size = input_tensor.shape[0]
        if self.evaluation: 
            mean = self.mean
            sigma = self.sigma
        else:            
            mean = input_tensor.mean(0)
            sigma = input_tensor.var(0, unbiased=False)
                        
            self.mean = self.mean + (mean - self.mean) * self.momentum            
            bessel_correction = batch_size / (batch_size - 1)
            self.sigma = self.sigma + (sigma * bessel_correction - self.sigma) * self.momentum
        # Напишите в этом месте нормирование входного тензора.
        normed_tensor = torch.zeros(input_tensor.shape)
        for n in range(batch_size):
            z = (input_tensor[n] - mean) / (sigma + eps).sqrt()
            normed_tensor[n] = z * self.weight + self.bias

        return normed_tensor

    def eval(self):
        self.evaluation = True


batch_norm = nn.BatchNorm1d(input_size, eps=eps)
batch_norm.bias.data = torch.randn(input_size, dtype=torch.float)
batch_norm.weight.data = torch.randn(input_size, dtype=torch.float)
batch_norm.momentum = 0.5

custom_batch_norm1d = CustomBatchNorm1d(batch_norm.weight.data,
                                        batch_norm.bias.data, eps, batch_norm.momentum)

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
all_correct = True

for i in range(8):
    torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
    norm_output = batch_norm(torch_input)
    custom_output = custom_batch_norm1d(torch_input)
    all_correct &= torch.allclose(norm_output, custom_output) \
        and norm_output.shape == custom_output.shape
print(all_correct)

batch_norm.eval()
custom_batch_norm1d.eval()

for i in range(8):
    torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
    norm_output = batch_norm(torch_input)
    custom_output = custom_batch_norm1d(torch_input)
    all_correct &= torch.allclose(norm_output, custom_output) \
        and norm_output.shape == custom_output.shape
print(all_correct)

True
True


In [16]:
# Случай многоканального тензора
import torch
import torch.nn as nn

eps = 1e-3

input_channels = 3
batch_size = 3
height = 10
width = 10

batch_norm_2d = nn.BatchNorm2d(input_channels, affine=False, eps=eps)

input_tensor = torch.randn(batch_size, input_channels, height, width, dtype=torch.float)


def custom_batch_norm2d(input_tensor, eps):
    mean, var = torch.tensor(input_tensor.numpy().mean(axis=(0,2,3)).reshape(1, input_tensor.shape[1], 1, 1)) \
              , torch.tensor(input_tensor.numpy().var( axis=(0,2,3) ).reshape(1, input_tensor.shape[1], 1, 1))

    return (input_tensor - mean) / torch.sqrt(var + eps)


# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
norm_output = batch_norm_2d(input_tensor)
custom_output = custom_batch_norm2d(input_tensor, eps)
print(torch.allclose(norm_output, custom_output) and norm_output.shape == custom_output.shape)

True


In [19]:
# Нормализация по каналу
# Идея, лежащая в основе слоя нормализации "по каналу", что сеть должна быть независимой от контраста исходного изображения.
import torch
import torch.nn as nn


eps = 1e-10


def custom_layer_norm(input_tensor, eps):
    normed_tens = torch.zeros(input_tensor.shape)
    for i in range (input_tensor.shape[0]):
        mean = torch.tensor(input_tensor[i].numpy().mean())
        var = torch.tensor(input_tensor[i].numpy().var())
        normed_tens[i] += (input_tensor[i] - mean) / torch.sqrt(var + eps)
    return normed_tens


# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
all_correct = True
for dim_count in range(3, 9):
    input_tensor = torch.randn(*list(range(3, dim_count + 2)), dtype=torch.float)
    layer_norm = nn.LayerNorm(input_tensor.size()[1:], elementwise_affine=False, eps=eps)

    norm_output = layer_norm(input_tensor)
    custom_output = custom_layer_norm(input_tensor, eps)

    all_correct &= torch.allclose(norm_output, custom_output, 1e-2)
    all_correct &= norm_output.shape == custom_output.shape
print(input_tensor.shape)

torch.Size([3, 4, 5, 6, 7, 8, 9])


In [21]:
# Нормализация "по инстансу" была изначально разработана для задачи style transfer.
# Идея, лежащая в основе этого слоя, что сеть должна быть независимой от контраста отдельных каналов исходного изображения.
import torch
import torch.nn as nn

eps = 1e-3

batch_size = 5
input_channels = 2
input_length = 30

instance_norm = nn.InstanceNorm1d(input_channels, affine=False, eps=eps)

input_tensor = torch.randn(batch_size, input_channels, input_length, dtype=torch.float)


def custom_instance_norm1d(input_tensor, eps):
    mean = input_tensor.mean(dim=2, keepdim=True)
    var = input_tensor.var(dim=2, unbiased=False, keepdim=True)
    return (input_tensor - mean) / torch.sqrt(var + eps) * 1 + 0


# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
norm_output = instance_norm(input_tensor)
custom_output = custom_instance_norm1d(input_tensor, eps)
print(torch.allclose(norm_output, custom_output) and norm_output.shape == custom_output.shape)

True


In [23]:
# Нормализация "по группе" - это обобщение нормализации "по каналу" и "по инстансу".
import torch
import torch.nn as nn

channel_count = 6
eps = 1e-3
batch_size = 20
input_size = 2

input_tensor = torch.randn(batch_size, channel_count, input_size)


def custom_group_norm(input_tensor, groups, eps):
    batch_size = input_tensor.shape[0]
    input_tensor_shape = input_tensor.shape
    input_tensor = input_tensor.reshape(batch_size, groups, -1)
    mean = input_tensor.mean(dim=2, keepdim=True)
    var = input_tensor.var(dim=2, unbiased=False, keepdim=True)
    normed_tensor = (input_tensor - mean) / torch.sqrt(var + eps) * 1 + 0
    return normed_tensor.reshape(*input_tensor_shape)


# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
all_correct = True
for groups in [1, 2, 3, 6]:
    group_norm = nn.GroupNorm(groups, channel_count, eps=eps, affine=False)
    norm_output = group_norm(input_tensor)
    custom_output = custom_group_norm(input_tensor, groups, eps)
    all_correct &= torch.allclose(norm_output, custom_output, 1e-3)
    all_correct &= norm_output.shape == custom_output.shape
print(all_correct)

True
